In [1]:
%reset -f
import h5py
import time as t
import numpy as np
import scipy as sp
import scipy.io as spi
import tensorflow as tf
import matplotlib.pyplot as plt

data_train = spi.loadmat('train_PDE_DR.mat')

u_in = data_train['X_train0'][0:50000,:]
x_t_in = data_train['X_train1'][0:50000,:]
s_in = data_train['y_train'][0:50000,:]

u_in.shape, x_t_in.shape, s_in.shape

((50000, 100), (50000, 2), (50000, 1))

In [2]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Lambda, Dense

bs = 50000

def fn(x):
    y = tf.einsum("ij, ij->i", x[0], x[1])
    y = tf.expand_dims(y, axis = 1)
    return y

hln = 25

inputsB = Input(shape = (100,), name = 'inputsB')
hiddenB = Dense(hln, activation = "relu")(inputsB)
hiddenB = Dense(hln, activation = "relu")(hiddenB)
hiddenB = Dense(hln, activation = "relu")(hiddenB)
hiddenB = Dense(hln, activation = "relu")(hiddenB)

inputsT = Input(shape = (2,), name = 'inputsT')
hiddenT = Dense(hln, activation = "relu")(inputsT)
hiddenT = Dense(hln, activation = "relu")(hiddenT)
hiddenT = Dense(hln, activation = "relu")(hiddenT)
hiddenT = Dense(hln, activation = "relu")(hiddenT)

combined = Lambda(fn, output_shape = [None, 1])([hiddenB, hiddenT])
output = Dense(1)(combined)

model = Model(inputs = [inputsB, inputsT], outputs = output)    

model.compile(optimizer = tf.optimizers.Adam(learning_rate = 0.001), loss = 'mse')
model.summary()

model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath = './ChkPts/CIII_V2_VDN_S1_10000/',
    save_weights_only=True,
    monitor='loss',
    mode='min',
    save_best_only=True)

itr = 0
for i in range(0, 40):
    print('Iteration: '+str(itr))
    itr = itr+1
    model.fit({"inputsB":u_in, "inputsT":x_t_in}, s_in, epochs = 1,
              verbose = 1, batch_size = bs, callbacks = [model_checkpoint_callback])    

    itr_ps = 250-1
    model.fit({"inputsB":u_in, "inputsT":x_t_in}, s_in, epochs = itr_ps,
              verbose = 0, batch_size = bs, callbacks = [model_checkpoint_callback])
    itr = itr+itr_ps

print('Total iterations: '+str(itr))

model.load_weights('./ChkPts/CIII_V2_VDN_S1_10000/')

model.save_weights('./model/CIII_V2_VDN_S1_10000')

# model.load_weights('./model/CIII_V2_VDN_S1')

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 inputsB (InputLayer)           [(None, 100)]        0           []                               
                                                                                                  
 inputsT (InputLayer)           [(None, 2)]          0           []                               
                                                                                                  
 dense (Dense)                  (None, 25)           2525        ['inputsB[0][0]']                
                                                                                                  
 dense_4 (Dense)                (None, 25)           75          ['inputsT[0][0]']                
                                                                                              

In [3]:
data_test = spi.loadmat('test_PDE_DR.mat')

u_in_test = data_test['X_test0']
x_t_in_test = data_test['X_test1']
s_in_test = data_test['y_test']

pred = model({"inputsB":u_in_test, "inputsT":x_t_in_test})
    
print()
print(np.mean((s_in_test-pred)**2))
print(np.mean((s_in_test)**2))
print(np.mean((s_in_test-pred)**2)/np.mean((s_in_test)**2))    


0.0021799998
0.23837758612778512
0.009145154421609878


In [4]:
# data_test = spi.loadmat('DR_Sample4.mat')

# u = data_test['force']
# sol = data_test['sol']

# x = np.linspace(0,1,100)
# t = np.linspace(0,1,100)

# x_t_in_stest = np.zeros([10000,2])
# u_in_stest = np.tile(u,[10000,1])

# for i in range(0,100):
#     for j in range(0,100):
#         x_t_in_stest[100*i+j,0] = x[i]
#         x_t_in_stest[100*i+j,1] = t[j]        
        
# u.shape

In [5]:
# pred = model({"inputsB":u_in_stest, "inputsT":x_t_in_stest})

In [6]:
# pred = pred.numpy().reshape([100,100])

In [7]:
# vmin = -0.75
# vmax = 1.25

# plt.imshow(pred, vmin = vmin, vmax = vmax)
# plt.xticks([0, 50, 100],[0, 0.5, 1], fontsize = 12)
# plt.yticks([0, 50, 100],[0, 0.5, 1], fontsize = 12)
# plt.xlabel('t', fontsize = 20)
# plt.ylabel('x', fontsize = 20)
# plt.colorbar()
# plt.tight_layout()
# plt.savefig('DR_VDN_PRED.pdf', bbox_inches = 'tight')
# plt.show()

# plt.imshow(sol, vmin = vmin, vmax = vmax)
# plt.xticks([0, 50, 100],[0, 0.5, 1], fontsize = 12)
# plt.yticks([0, 50, 100],[0, 0.5, 1], fontsize = 12)
# plt.xlabel('t', fontsize = 20)
# plt.ylabel('x', fontsize = 20)
# plt.colorbar()
# plt.tight_layout()
# plt.savefig('DR_GT.pdf', bbox_inches = 'tight')
# plt.show()

# plt.imshow(np.abs(pred-sol), vmin = 0, vmax = 0.15)
# plt.xticks([0, 50, 100],[0, 0.5, 1], fontsize = 12)
# plt.yticks([0, 50, 100],[0, 0.5, 1], fontsize = 12)
# plt.colorbar()
# plt.tight_layout()
# plt.savefig('DR_VDN_ERROR.pdf', bbox_inches = 'tight')
# plt.show()